In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from ipynb.fs.full.project2 import properties
from ipynb.fs.full.project3_functions import i_sr, di_s, K_inc, K_M, K_p, K_Re, Y_p1, Y_p2
from ipynb.fs.full.project3_functions import Y_p, Y_s, Y_TE, Y_sh, Y_EX, Y_CL, Y_LW
from ipynb.fs.full.project3_functions import dh0_leak, dh0_DF, dh0_adm, dh0_gap

# Non-Dimensional Parameters

In [2]:
def ND_Parameters(psi_c, phi_c, delta, R_c, r_c):
    phi_opt = 0.375+0.25*psi_c
    etta_opt = 0.913+0.103*psi_c-0.0854*psi_c**2+0.0154*psi_c**3

    if psi_c <= 2.8:
        etta_opt = 0.913+0.103*psi_c-0.0854*psi_c**2+0.0154*psi_c**3
    
    else:
        etta_opt = 1.01-0.05*psi_c
    
    
    if psi_c <= 2.2:
        K = 0.375-0.125*psi_c
    
    else:
        K = 0.22/psi_c
    
    etta_o = etta_opt-K*(phi_c-phi_opt)**2
    
    psi_opt = 1.35*phi_c**1.2
    
    h_bx = 0.99*1/(0.4+0.5*R_c)
    
    
    h1 = 2*r_c
    
    K = delta/(r_c+h1/2)
    
    X = h_bx*(0.4+0.5*R_c)
    
    etta = etta_opt*(1-0.93*K*(r_c/h1)*(1+h1/(2*r_c))**2)*X**(0.05+0.04*R_c)
    
    dic = {
        'psi_opt' : psi_opt,
        'phi_opt' : phi_opt,
        'etta_opt' : etta_opt,
        'etta_o' : etta_o,
        'etta' : etta,
    }
    
    return dic

# Velocity Triangles

In [3]:
def Velocity_Triangles(U_c, phi_c, psi_c, R_c, alpha1_c, alpha4_c=0):
    Cm1_c = phi_c*U_c
    
    if alpha4_c != 0:
        C_theta4_c = phi_c/math.tan(math.radians(alpha4_c))*U_c
        C_theta3_c = (psi_c+phi_c/math.tan(math.radians(alpha4_c)))*U_c
        
    else:
        C_theta3_c = U_c*(1-R_c+(psi_c/2))
        C_theta4_c = U_c*(1-R_c-(psi_c/2))
        
    
    
    C_theta2_c = C_theta3_c
    
    alpha3_c = math.degrees(math.atan(Cm1_c/C_theta3_c))
    alpha2_c = alpha3_c
    alpha4_c = math.degrees(math.atan(Cm1_c/C_theta4_c))
    
    C_theta1_c = phi_c/math.tan(math.radians(alpha1_c))*U_c
    
    W_theta3_c = C_theta3_c-U_c
    W_theta2_c = W_theta3_c
    W_theta4_c = C_theta4_c-U_c
    
    Beta3_c = math.degrees(math.atan(Cm1_c/W_theta3_c))
    Beta2_c = Beta3_c
    Beta4_c = math.degrees(math.atan(Cm1_c/W_theta4_c))
    
    point1 = {
        'C_theta[meter/sec]' : C_theta1_c,
        'Cm[meter/sec]'      : Cm1_c,
        'C[meter/sec]'       : abs(complex(Cm1_c, C_theta1_c)),
        'alpha[degree]'      : alpha1_c,
        'W_theta[meter/sec]' : '-',
        'W[meter/sec]'       : '-',
        'Beta[degree]'       : '-',
    }
    
    point2 = {
        'C_theta[meter/sec]' : C_theta2_c,
        'Cm[meter/sec]'      : Cm1_c,
        'C[meter/sec]'       : abs(complex(Cm1_c, C_theta2_c)),
        'alpha[degree]'      : alpha2_c,
        'W_theta[meter/sec]' : W_theta2_c,
        'W[meter/sec]'       : abs(complex(Cm1_c, W_theta2_c)),
        'Beta[degree]'       : Beta2_c,
    }
    
    point3 = {
        'C_theta[meter/sec]' : C_theta3_c,
        'Cm[meter/sec]'      : Cm1_c,
        'C[meter/sec]'       : abs(complex(Cm1_c, C_theta3_c)),
        'alpha[degree]'      : alpha3_c,
        'W_theta[meter/sec]' : W_theta3_c,
        'W[meter/sec]'       : abs(complex(Cm1_c, W_theta3_c)),
        'Beta[degree]'       : Beta3_c,
    }
    
    point4 = {
        'C_theta[meter/sec]' : C_theta4_c,
        'Cm[meter/sec]'      : Cm1_c,
        'C[meter/sec]'       : abs(complex(Cm1_c, C_theta4_c)),
        'alpha[degree]'      : alpha4_c,
        'W_theta[meter/sec]' : W_theta4_c,
        'W[meter/sec]'       : abs(complex(Cm1_c, W_theta4_c)),
        'Beta[degree]'       : Beta4_c,
    }
    
    Properties = {
    'Point 1' : point1,
    'Point 2' : point2,
    'Point 3' : point3,     
    'Point 4' : point4
    }
    
    df = pd.DataFrame(Properties).T
    
    return df

# Number of Blades

In [4]:
def NOB(Beta1, Beta2, Beta1_prime, row, c, r_c):
    #A&M Method:
    s_c0 = 0.427+Beta2/58-(Beta2/93)**2
    s_c1 = 0.224+((1.575-Beta2)/90)*(Beta2/90)
    landa = (90-Beta1_prime)/(90-Beta2)
    s_c_AM = s_c0+(s_c1-s_c0)*abs(landa)*landa
    
    #ZW Method:
    landa = 0.9
    cot_Beta_m =(1/math.tan(math.radians(Beta2))-1/math.tan(math.radians(Beta1)))/2
    Beta_m = math.degrees(math.asin(1/((1+cot_Beta_m**2)**0.5)))
    A = (math.sin(math.radians(Beta2))**2)/math.sin(math.radians(Beta_m))
    C = 2*A*(1/math.tan(math.radians(Beta2))+1/math.tan(math.radians(Beta1)))
    s_c_zw = landa/C
    
    s_AM = s_c_AM*c
    s_zw = s_c_zw*c
    
    NOB_AM = 2*math.pi*r_c/s_AM
    NOB_zw = 2*math.pi*r_c/s_zw
    
    if row == 'Stator':
        if (math.ceil(NOB_AM)%2) == 0:
            NOB_AM = math.ceil(NOB_AM)
        
        else:
            NOB_AM = math.ceil(NOB_AM)+1
        
        if (math.ceil(NOB_zw)%2) == 0:
            NOB_zw = math.ceil(NOB_zw)
        
        else:
            NOB_zw = math.ceil(NOB_zw)+1
        
    else:
        if (math.ceil(NOB_AM)%2) != 0:
            NOB_AM = math.ceil(NOB_AM)
        
        else:
            NOB_AM = math.ceil(NOB_AM)+1
        
        if (math.ceil(NOB_zw)%2) != 0:
            NOB_zw = math.ceil(NOB_zw)
        
        else:
            NOB_zw = math.ceil(NOB_zw)+1
            
    return {'NOB_AM' : NOB_AM, 's_AM' : s_AM, 'NOB_zw' : NOB_zw, 's_zw' : s_zw}

# Velocity Triangles from Hub to Shroud

## Section 1

In [5]:
def Section1(r1, Cm1_c, alpha1, R, Properties01, Properties1):
    Cm = Cm1_c
    C_theta = Cm/math.tan(math.radians(alpha1))
    
    P0  = Properties01['P[bar]']
    T0  = Properties01['T[k]']
    h0  = Properties01['h[kj/kg]']
    s   = Properties01['s[kj/kg-k]']
    P   = Properties1['P[bar]']
    T   = Properties1['T[k]']
    h   = Properties1['h[kj/kg]']
    rho = P*101.325/(R*T)
    
    dic = {
        'P0[bar]'     : P0,
        'T0[k]'       : T0,
        'h0[kj/kg]'   : h0,
        's[kj/kg-k]'  : s,
        'P[bar]'      : P,
        'T[k]'        : T,
        'h[kj/kg]'    : h,
        'rho[kg/m^3]' : rho,
    }
    
    return dic

## Vortex Function

In [6]:
def Reaction(r_c, U_c, R_c, psi_c, r, omega, C_theta2_c, alpha2_c, C3, C4, vortex = 'Vortex'):
    if vortex == 'FV':
        R = 1-(1-R_c)*(r_c/r)**2
        
    elif vortex == 'CSV':
        R = R_c-(2*(1-R_c)-psi_c*(r_c/r-1)/2)*(r_c/r-1)
        
    elif vortex == 'EV':
        R = R_c-2*(1-R_c)*(r_c/r-1)
        
    elif vortex == 'CRV':
        R = R_c-2*(1-R_c)*np.log(r_c/r)
        
    elif vortex == 'CNAV':
            R = 1-(C3**2-C4**2)/(2*(U_c**2)*psi_c)

    return R

In [7]:
def C_theta_2(r_c, U_c, R_c, psi_c, r, omega, C_theta2_c, alpha2_c, vortex = 'Vortex'):
    if vortex == 'FV':
        C_theta3 = (r_c*U_c*(1-R_c+psi_c/2))/r
        
    elif vortex == 'CSV':
        C_theta3 = U_c*(1-R_c+psi_c/2)
        
    elif vortex == 'EV':
        C_theta3 = U_c*(1-R_c+(psi_c/2)*(r_c/r)) 
        
    elif vortex == 'CRV':
        C_theta3 = r*omega*(1-R_c+(psi_c/2)*(r_c/r)**2)

        
    elif vortex == 'CNAV':
        C_theta3 = C_theta2_c*(r/r_c)**(-(math.cos(math.radians(alpha2_c)))**2)
        
    return C_theta3

In [8]:
def C_theta_4(r_c, U_c, R_c, psi_c, r, omega, C_theta3, vortex = 'Vortex'):
    if vortex == 'FV':
        C_theta4 = (r_c*U_c*(1-R_c-psi_c/2))/r
        
    elif vortex == 'CSV':
        C_theta4 = C_theta3-U_c*psi_c*r_c/r
        
    elif vortex == 'EV':
        C_theta4 = U_c*(1-R_c-(psi_c/2)*(r_c/r)) 
        
    elif vortex == 'CRV':
        C_theta4 = r*omega*(1-R_c-(psi_c/2)*(r_c/r)**2)

        
    elif vortex == 'CNAV':
        C_theta4 = C_theta3-psi_c*(r_c/r)*U_c
        
    return C_theta4

## Sizing

## Section 2

In [9]:
def radius_section2(C2_c, alpha2_c, delta_r, r_c, rho2_c, M2_c, h02_c, s2_c, r2, n, p, R, R_c, psi_c, U_c, side, omega, C_theta2_c, Cm1_c, rho1_c, r1, i, h1_c, v = 'Vortex'):
    itteration = 1
    difference = 1
    limit = 10e-3
    
    if side == 'upside':
        step = 0.0001
        
    else:
        step = -0.0001
    
    while (difference > limit):
        delta_C = C2_c*(-(math.cos(math.radians(alpha2_c)))**2)*delta_r/r_c
        C2 = C2_c+delta_C

        delta_rho = rho2_c*(M2_c**2)*((math.cos(math.radians(alpha2_c)))**2)*delta_r/r_c
        rho2 = rho2_c+delta_rho

        h2 = h02_c-C2**2/2000 
    
        if h2 > h1_c:
            r2[i] = r2[i]+step
            delta_r = r2[i]-r_c
            itteration += 1
            continue
        
        
        s2 = s2_c
        dic = properties({'h' : h2, 's' : s2})
        T2 = dic['T[k]']
        P2 = dic['P[bar]']
        C_p = dic['C_p[kj/kg-k]']
        gama2 = C_p/(C_p-R)

        C_theta2 = C_theta_2(r_c, U_c, R_c, psi_c, r2[i], omega, C_theta2_c, alpha2_c, vortex = v)

        if ((abs(C_theta2/C2) <= 1) & (abs(C_theta2/C2) >= 0)):
            alpha2 = math.degrees(math.acos(C_theta2/C2))
            Cm2 = C2*math.sin(math.radians(alpha2))
            M2 = C2/((gama2*R*1000*T2)**0.5)

        else:
            C_theta2_C2 = C_theta2/C2
            if C_theta2_C2 > 1:
                C_theta2_C2 = math.floor(C_theta2_C2)
            elif C_theta2_C2 < -1:
                C_theta2_C2 = math.ceil(C_theta2_C2)
                       
            alpha2 = math.degrees(math.acos(C_theta2_C2))
            Cm2 = C2*math.sin(math.radians(alpha2))
            M2 = C2/((gama2*R*1000*T2)**0.5)
    
        if (i == p+1) or (i == p-1):
            if side == 'upside':
                dm_dot1 = rho1_c*math.pi*(r1[i]**2-r_c**2)*Cm1_c
                dm_dot2 = rho2*math.pi*(r2[i]**2-r_c**2)*Cm2 
        
            else:
                dm_dot1 = rho1_c*math.pi*(r_c**2-r1[i]**2)*Cm1_c
                dm_dot2 = rho2*math.pi*(r_c**2-r2[i]**2)*Cm2
        
        else:
            if side == 'upside':
                dm_dot1 = rho1_c*math.pi*(r1[i]**2-r1[i-1]**2)*Cm1_c
                dm_dot2 = rho2*math.pi*(r2[i]**2-r2[i-1]**2)*Cm2 
        
            else:
                dm_dot1 = rho1_c*math.pi*(r1[i+1]**2-r1[i]**2)*Cm1_c
                dm_dot2 = rho2*math.pi*(r2[i+1]**2-r2[i]**2)*Cm2
        
        difference = abs(dm_dot1-dm_dot2)
    
        if (dm_dot2 < 0) or (dm_dot2 == 0):
            r2[i] = r2[i]+step
            delta_r = r2[i]-r_c
            itteration += 1
            continue
    
        r2[i] = r2[i]+step
        delta_r = r2[i]-r_c
        itteration += 1
        
        if itteration > 300:
            print('The Method is not Appropriate')
        
    #print(itteration)  
    return r2[i], h2, s2, P2, T2, rho2, C_theta2, Cm2, alpha2, M2

In [10]:
def Specifications2(m, C2_c, alpha2_c, r_c, h02_c, n, R, R_c, psi_c, U_c, omega, C_theta2_c, Cm1_c, rho1_c, r1, h1_c, h2_c, s2_c, P2_c, T2_c, rho2_c, Cm2_c, M2_c, Vortex):
    
    r2 = r1
    p = m-1
    r2 = list(r2)
    
    h2          = list(range(0, n))
    s2          = list(range(0, n))
    P2          = list(range(0, n))
    T2          = list(range(0, n))
    rho2        = list(range(0, n))
    C_theta2    = list(range(0, n))
    Cm2         = list(range(0, n))
    alpha2      = list(range(0, n))
    M2          = list(range(0, n))
    
    h2[p]       = h2_c
    s2[p]       = s2_c
    P2[p]       = P2_c
    T2[p]       = T2_c
    rho2[p]     = P2_c*101.325/(R*T2_c)
    C_theta2[p] = C_theta2_c
    Cm2[p]      = Cm2_c
    alpha2[p]   = alpha2_c
    M2[p]       = M2_c
    
    for index in range(1, m):
        side = 'upside'
        i = p+index
        r2[i] = r2[i-1]
        delta_r = r2[i]-r_c
    
    
        r2[i], h2[i], s2[i], P2[i], T2[i], rho2[i], C_theta2[i], Cm2[i], alpha2[i], M2[i] = radius_section2(
            C2_c, alpha2_c, delta_r, r_c, rho2_c, M2_c, h02_c, s2_c,
            r2, n, p, R, R_c, psi_c, U_c, side, omega, C_theta2_c,
            Cm1_c, rho1_c, r1, i, h1_c, v = Vortex
        )
    
    for index in range(1, m):
        side = 'downside'
        i = p-index
        r2[i] = r2[i+1]
        delta_r = r2[i]-r_c
    
        r2[i], h2[i], s2[i], P2[i], T2[i], rho2[i], C_theta2[i], Cm2[i], alpha2[i], M2[i] = radius_section2(
            C2_c, alpha2_c, delta_r, r_c, rho2_c, M2_c, h02_c, s2_c,
            r2, n, p, R, R_c, psi_c, U_c, side, omega, C_theta2_c,
            Cm1_c, rho1_c, r1, i, h1_c, v = Vortex
        )
    
    r2          = np.array(r2)
    h2          = np.array(h2)
    s2          = np.array(s2)
    P2          = np.array(P2)
    T2          = np.array(T2)
    rho2        = np.array(rho2)
    C_theta2    = np.array(C_theta2)
    Cm2         = np.array(Cm2)
    alpha2      = np.array(alpha2)
    M2          = np.array(M2)
    
    h_blade2 = r2[-1]-r2[0]
    
    return r2, h2, s2, P2, T2, rho2, C_theta2, Cm2, alpha2, M2, h_blade2

## Section 4

In [11]:
def radius_section4(Cm3, rho3, C_theta3, C4_c, alpha4_c, delta_r, r_c, rho4_c, M4_c, h04_c, s4_c, r4, n, p, R, R_c, psi_c, U_c, side, omega, r3, i, h3_c, v = 'Vortex'):
    itteration = 1
    difference = 1
    limit = 10e-3
    
    if side == 'upside':
        step = 0.0001
        
    else:
        step = -0.0001
    
    while (difference > limit):
        
        if side == 'upside':
            
            C_theta4_r  = C_theta_4(r_c, U_c, R_c, psi_c, r4[i], omega, C_theta3[i], vortex = v)
            C_theta4_rp = C_theta_4(r_c, U_c, R_c, psi_c, r4[i-1], omega, C_theta3[i-1], vortex = v)
            
            C_theta3_r  = C_theta3[i]
            C_theta3_rp = C_theta3[i-1]
            
            U4_r  = r4[i]*omega
            U4_rp = r4[i-1]*omega
            
            U3_r  = r3[i]*omega
            U3_rp = r3[i-1]*omega
            
            r4_r  = r4[i]
            r4_rp = r4[i-1]
            
            r3_r  = r3[i]
            r3_rp = r3[i-1]
            
        else:
            C_theta4_r = C_theta_4(r_c, U_c, R_c, psi_c, r4[i], omega, C_theta3[i], vortex = v)
            C_theta4_rp = C_theta_4(r_c, U_c, R_c, psi_c, r4[i+1], omega, C_theta3[i+1], vortex = v)
            
            C_theta3_r = C_theta3[i]
            C_theta3_rp = C_theta3[i+1]
            
            U4_r  = r4[i]*omega
            U4_rp = r4[i+1]*omega
            
            U3_r  = r3[i]*omega
            U3_rp = r3[i+1]*omega
            
            r4_r  = r4[i]
            r4_rp = r4[i+1]
            
            r3_r  = r3[i]
            r3_rp = r3[i+1]
            
        if r4_r != r4_rp:
            dh04_dr = -(U3_r*C_theta3_r-U3_rp*C_theta3_rp)/(r3_r-r3_rp)+(U4_r*C_theta4_r-U4_rp*C_theta4_rp)/(r4_r-r4_rp)
        else:
            r4[i] = r4[i]+step
            delta_r = r4[i]-r_c
            itteration += 1
            continue
            

        delta_C = (delta_r/C4_c)*dh04_dr-C4_c*((math.cos(math.radians(alpha4_c)))**2)*delta_r/r_c
        C4 = C4_c+delta_C

        delta_rho = rho4_c*(M4_c**2)*((math.cos(math.radians(alpha4_c)))**2)*delta_r/r_c
        rho4 = rho4_c+delta_rho
        
        h04 = h04_c+dh04_dr*delta_r/1000
        h4 = h04_c-C4**2/2000 
    
        if h4 > h3_c:
            r4[i] = r4[i]+step
            delta_r = r4[i]-r_c
            itteration += 1
            continue
        
        
        s4 = s4_c
        dic = properties({'h' : h4, 's' : s4})
        T4 = dic['T[k]']
        P4 = dic['P[bar]']
        C_p = dic['C_p[kj/kg-k]']
        gama4 = C_p/(C_p-R)
        
        C_theta4 = C_theta4_r

        if ((abs(C_theta4/C4) <= 1) & (abs(C_theta4/C4) >= 0)):
            alpha4 = math.degrees(math.acos(C_theta4/C4))
            Cm4 = C4*math.sin(math.radians(alpha4))
            M4 = C4/((gama4*R*1000*T4)**0.5)

        else:
            C_theta4_C4 = C_theta4/C4
            if C_theta4_C4 > 1:
                C_theta4_C4 = math.floor(C_theta4_C4)
            elif C_theta4_C4 < -1:
                C_theta4_C4 = math.ceil(C_theta4_C4)
                       
            alpha4 = math.degrees(math.acos(C_theta4_C4))
            Cm4 = C4*math.sin(math.radians(alpha4))
            M4 = C4/((gama4*R*1000*T4)**0.5)
    
        if (i == p+1) or (i == p-1):
            if side == 'upside':
                dm_dot3 = rho3*math.pi*(r3[i]**2-r_c**2)*Cm3
                dm_dot4 = rho4*math.pi*(r4[i]**2-r_c**2)*Cm4 
        
            else:
                dm_dot3 = rho3*math.pi*(r_c**2-r3[i]**2)*Cm3
                dm_dot4 = rho4*math.pi*(r_c**2-r4[i]**2)*Cm4
        
        else:
            if side == 'upside':
                dm_dot3 = rho3*math.pi*(r3[i]**2-r3[i-1]**2)*Cm3
                dm_dot4 = rho4*math.pi*(r4[i]**2-r4[i-1]**2)*Cm4 
        
            else:
                dm_dot3 = rho3*math.pi*(r3[i+1]**2-r3[i]**2)*Cm3
                dm_dot4 = rho4*math.pi*(r4[i+1]**2-r4[i]**2)*Cm4
        
        difference = abs(dm_dot3-dm_dot4)
    
        if (dm_dot4 < 0) or (dm_dot4 == 0):
            r4[i] = r4[i]+step
            delta_r = r4[i]-r_c
            itteration += 1
            continue
    
        r4[i] = r4[i]+step
        delta_r = r4[i]-r_c
        itteration += 1
        
        if itteration > 300:
            print('The Method is not Appropriate')
          
    return r4[i], h4, h04, s4, P4, T4, rho4, C_theta4, Cm4, alpha4, M4

In [12]:
def Specifications4(Cm3, rho3, C_theta3, m, C4_c, alpha4_c, r_c, n, R, R_c, psi_c, U_c, omega, C_theta4_c, r3, h3_c, h4_c, h04_c, s4_c, P4_c, T4_c, rho4_c, Cm4_c, M4_c, Vortex):
    r4 = r3
    p = m-1
    r4 = list(r4)
    
    h4          = list(range(0, n))
    h04         = list(range(0, n))
    s4          = list(range(0, n))
    P4          = list(range(0, n))
    T4          = list(range(0, n))
    rho4        = list(range(0, n))
    C_theta4    = list(range(0, n))
    Cm4         = list(range(0, n))
    alpha4      = list(range(0, n))
    M4          = list(range(0, n))
    
    h4[p]       = h4_c
    h04[p]      = h04_c
    s4[p]       = s4_c
    P4[p]       = P4_c
    T4[p]       = T4_c
    rho4[p]     = P4_c*101.325/(R*T4_c)
    C_theta4[p] = C_theta4_c
    Cm4[p]      = Cm4_c
    alpha4[p]   = alpha4_c
    M4[p]       = M4_c
    
    for index in range(1, m):
        side = 'upside'
        i = p+index
        r4[i] = r4[i-1]
            
        delta_r = r4[i]-r_c
    
    
        r4[i], h4[i], h04[i], s4[i], P4[i], T4[i], rho4[i], C_theta4[i], Cm4[i], alpha4[i], M4[i] = radius_section4(
            Cm3[i], rho3[i], C_theta3, C4_c, alpha4_c, delta_r, r_c,
            rho4_c, M4_c, h04_c,s4_c, r4, n, p, R, R_c, psi_c,
            U_c, side, omega, r3, i, h3_c, v = Vortex
        )
    
    for index in range(1, m):
        side = 'downside'
        i = p-index
        r4[i] = r4[i+1]
            
        delta_r = r4[i]-r_c
    
        r4[i], h4[i], h04[i], s4[i], P4[i], T4[i], rho4[i], C_theta4[i], Cm4[i], alpha4[i], M4[i] = radius_section4(
            Cm3[i], rho3[i], C_theta3, C4_c, alpha4_c, delta_r, r_c,
            rho4_c, M4_c, h04_c,s4_c, r4, n, p, R, R_c, psi_c,
            U_c, side, omega, r3, i, h3_c, v = Vortex
        )
    
    r4          = np.array(r4)
    h4          = np.array(h4)
    h04         = np.array(h04)
    s4          = np.array(s4)
    P4          = np.array(P4)
    T4          = np.array(T4)
    rho4        = np.array(rho4)
    C_theta4    = np.array(C_theta4)
    Cm4         = np.array(Cm4)
    alpha4      = np.array(alpha4)
    M4          = np.array(M4)
    
    h_blade4 = r4[-1]-r4[0]
    
    return r4, h4, h04, s4, P4, T4, rho4, C_theta4, Cm4, alpha4, M4, h_blade4